In [9]:
# from ps_algorithm import PSAlgorithm
# from svg_converter import SVGConverter
# from camera import Camera

%run /home/xilinx/jupyter_notebooks/pen_plotter/camera.ipynb
%run /home/xilinx/jupyter_notebooks/pen_plotter/svg_converter.ipynb
%run /home/xilinx/jupyter_notebooks/pen_plotter/ps_algorithm.ipynb
%run /home/xilinx/jupyter_notebooks/pen_plotter/pen_plotter.ipynb
# %run /home/xilinx/jupyter_notebooks/pen_plotter/background_removal.ipynb
from PIL import Image


class MainController:


  def __init__(self) -> None:
    self.hardware_acceleration = False
    self.camera = Camera()
    self.ps_algorithm = PSAlgorithm()
    self.svg_converter = SVGConverter()
    self.plotter = PenPlotter()
    self.raw_image : Image

  def enable_hardware_acceleration(self) -> None:
    self.hardware_acceleration = True

  def disable_hardware_acceleration(self) -> None:
    self.hardware_acceleration = False

  def clear(self) -> None:
    self.raw_image = None
    self.processed_svg = None

  def take_picture(self) -> Image:
    self.raw_image = self.camera.take_picture()
    return self.raw_image

  def process_image(self) -> str:
        
        if (self.hardware_acceleration):
          return ""
        else:
            width, height, points = self.ps_algorithm.process_image(self.raw_image)
            self.processed_svg, self.processed_png = self.svg_converter.fromPoints(width, height, points)
            return self.processed_svg, self.processed_png
        
  def plot_image(self) -> None:
        self.plotter.plot(self.processed_svg)



In [10]:
import numpy as np
import cv2 as cv
import cairosvg as cs
import imageio
import matplotlib.pyplot as plt
from PIL import Image
from time import sleep

print('importing base overlay')
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
base = BaseOverlay("base.bit")

print('connecting display')
frame_in_w = 640
frame_in_h = 480
Mode = VideoMode(frame_in_w,frame_in_h,24,fps=30)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start() 
print('display connected')
        
def display(img):
    
    new_size = (frame_in_w,frame_in_h)
    img = img.resize(new_size)
    img = np.array(img)[:, :, ::-1]
    #print(img[0:10])
    print('Sending image to Display')
    outframe = hdmi_out.newframe()
    outframe[:] = img
    hdmi_out.writeframe(outframe)
    print('Image Displayed')
    
image_path = 'data/Buttons_Page.png'
image = Image.open(image_path)
display(image)
    
    


importing base overlay
connecting display
display connected
Sending image to Display
Image Displayed


In [11]:
controller = MainController()
# controller.disable_hardware_acceleration()
# raw_image = controller.take_picture()
# image = controller.process_image()
# image



capture = 0
plot = 0

# for led in base.leds:
#     led.toggle()
#     sleep(0.1)

# for led in reversed(base.leds):
#     led.toggle()
#     sleep(0.1)
print('Connection established')
i=0
while (base.buttons[3].read()==0 and base.buttons[2].read()==0):
    if i==0:
        print('Inside while')
        i+=1
        
    if (base.buttons[0].read()==1):
        print('Button 0 is pressed')
        image_path = 'data/Buttons_Page.png'
        image = Image.open(image_path)
        display(image)
        
    elif (base.buttons[1].read()==1):
        capture = 1
        plot = 0
        print('signal sent to capture the image')
        pg2 = controller.take_picture()

        display(pg2)
        print(capture)

    elif (capture==1 and base.buttons[2].read()==1):
        plot = 1
        
        print('Button 2 is pressed')
            
        x=2
        print("image sent for processing")
        svg_path, pg3 = controller.process_image()
        print(svg_path)
        display(pg3)
        print('Image Displayed')


#         elif (base.switches[1].read()==0):
#             x=3
#             plot = 1
# #         Background removal algorithm
# #             pg3 = output from algorithm
#         display(pg3)
        

    
    elif (capture==1 and plot==1 and base.buttons[3].read()==1):
        capture = 0
        plot = 0
#         run the plotter

Connection established
Inside while
signal sent to capture the image
Sending image to Display
Image Displayed
1


In [4]:
hdmi_out.stop()